# Quickstart notebook
The example code here shows how to get up and running with Mosaic using the Python API.

In [0]:
from pyspark.sql.functions import *

## Enable Mosaic in the notebook
To get started, you'll need to attach the python library to your cluster and execute the `enable_mosaic` function.

In [0]:
from mosaic import enable_mosaic
enable_mosaic(spark)

Mosaic has extra configuration options. Check the docs for more details.

In [0]:
help(enable_mosaic)

Help on function enable_mosaic in module mosaic.functions:

enable_mosaic(spark: pyspark.sql.session.SparkSession) -> None
 Enable Mosaic functions.
 
 Use this function at the start of your workflow to ensure all of the required dependencies are installed and
 Mosaic is configured according to your needs.
 
 Parameters
 ----------
 spark : pyspark.sql.SparkSession
 The active SparkSession.
 
 Returns
 -------
 
 Notes
 -----
 Users can control various aspects of Mosaic's operation with the following Spark confs:
 
 - `spark.databricks.mosaic.jar.autoattach`: 'true' (default) or 'false'
 Automatically attach the Mosaic JAR to the Databricks cluster? (Optional)
 - `spark.databricks.mosaic.jar.location`
 Explicitly specify the path to the Mosaic JAR.
 (Optional and not required at all in a standard Databricks environment).
 - `spark.databricks.mosaic.geometry.api`: 'OGC' (default) or 'JTS'
 Explicitly specify the underlying geometry library to use for spatial operations. (Optional)
 - `spark.databricks.mosaic.index.system`: 'H3' (default)
 Explicitly specify the index system to use for optimized spatial joins. (Optional)

## Geometry constructors and the Mosaic internal geometry format

Mosaic allows users to create new Point geometries from a pair of Spark DoubleType columns.

In [0]:
from mosaic import st_point

lons = [-80., -80., -70., -70., -80.]
lats = [ 35.,  45.,  45.,  35.,  35.]

bounds_df = (
  spark
  .createDataFrame({"lon": lon, "lat": lat} for lon, lat in zip(lons, lats))
  .coalesce(1)
  .withColumn("point_geom", st_point("lon", "lat"))
)
bounds_df.show()

+----+-----+--------------------+
 lat| lon| point_geom|
+----+-----+--------------------+
35.0|-80.0|{1, [[[-80.0, 35....|
45.0|-80.0|{1, [[[-80.0, 45....|
45.0|-70.0|{1, [[[-70.0, 45....|
35.0|-70.0|{1, [[[-70.0, 35....|
35.0|-80.0|{1, [[[-80.0, 35....|
+----+-----+--------------------+

Mosaic Point geometries can be aggregated into LineString and Polygon geometries using the respective constructors.

In [0]:
from mosaic import st_makeline

bounds_df = (
  bounds_df
  .groupBy()
  .agg(collect_list("point_geom").alias("bounding_coords"))
  .select(st_makeline("bounding_coords").alias("bounding_ring"))
)
bounds_df.show()

+--------------------+
 bounding_ring|
+--------------------+
{3, [[[-80.0, 35....|
+--------------------+

In [0]:
from mosaic import st_makepolygon

bounds_df = bounds_df.select(st_makepolygon("bounding_ring").alias("bounds"))
bounds_df.show()

+--------------------+
 bounds|
+--------------------+
{5, [[[-80.0, 35....|
+--------------------+

## Geometry clipping without an index

Mosaic implements set intersection functions: contains, intersects, overlaps etc. Here you can see `st_contains` being used to clip points by a polygon geometry.

In [0]:
tripsTable = spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")

In [0]:
from mosaic import st_contains
trips = (
  tripsTable
  .limit(5_000_000)
  .repartition(sc.defaultParallelism * 20)
  .drop("vendorId", "rateCodeId", "store_and_fwd_flag", "payment_type")
  .withColumn("pickup_geom", st_point("pickup_longitude", "pickup_latitude"))
  .withColumn("dropoff_geom", st_point("dropoff_longitude", "dropoff_latitude"))
  .crossJoin(bounds_df)
  .where(st_contains("bounds", "pickup_geom"))
  .where(st_contains("bounds", "dropoff_geom"))
  .cache()
)

In [0]:
trips.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
 CMT|2009-05-16 23:09:38|2009-05-16 23:12:36| 1| 0.4| -73.993708| 40.724551| null| -74.001403| 40.730863| 3.7| 0.0| null| 0.0| 0.0| 3.7|{1, [[[-73.993708...|{1, [[[-74.001403...|{5, [[[-80.0, 35....|
 VTS|2010-04-30 05:50:00|2010-04-30 05:51:00| 1| 0.0| -74.183187| 40.68801| 5| -74.183145| 40.687972| 65.0| 0.0| 0.0| 20.0| 0.0| 85.0|{1, [[[-74.183187...|{1, [[[-74.183145...|{5, [[[-80.0, 35....|
 DDS|2009-05-29 17:25:37|2009-05-29 17:50:13| 1| 4.5| -74.002906| 40.728279| null| -73.954818| 40.767097| 15.3| 1.0| null| 4.05| 0.0| 20.35|{1, [[[-74.002906...|{1, [[[-73.954818...|{5, [[[-80.0, 35....|
 CMT|2009-12-28 18:19:23|2009-12-28 18:33:29| 1| 4.7| -73.997848| 40.726077| null| -73.988747| 40.777933| 13.3| 1.0| 0.5| 2.22| 0.0| 17.02|{1, [[[-73.997848...|{1, [[[-73.988747...|{5, [[[-80.0, 35....|
 CMT|2010-04-30 22:44:29|2010-04-30 22:57:05| 2| 2.5| -74.00514| 40.718808| 1| -73.986013| 40.751643| 9.3| 0.5| 0.5| 0.0| 0.0| 10.3|{1, [[[-74.00514,...|{1, [[[-73.986013...|{5, [[[-80.0, 35....|
 DDS|2009-05-03 00:12:28|2009-05-03 00:18:40| 1| 1.0| -73.991796| 40.725481| null| -73.977891| 40.72679| 5.3| 0.5| null| 0.0| 0.0| 5.8|{1, [[[-73.991796...|{1, [[[-73.977891...|{5, [[[-80.0, 35....|
 CMT|2009-01-31 01:36:33|2009-01-31 01:54:19| 1| 4.6| -73.990985| 40.734834| null| -73.957354| 40.726189| 15.0| 0.0| null| 0.0| 0.0| 15.0|{1, [[[-73.990985...|{1, [[[-73.957354...|{5, [[[-80.0, 35....|
 VTS|2010-02-09 00:39:00|2010-02-09 00:47:00| 1| 1.84| -74.004577| 40.723885| 1| -73.978537| 40.726283| 7.3| 0.5| 0.5| 0.0| 0.0| 8.3|{1, [[[-74.004577...|{1, [[[-73.978537...|{5, [[[-80.0, 35....|
 CMT|2010-02-05 17:00:49|2010-02-05 17:09:30| 1| 1.0| -73.999779| 40.718208| 1| -74.00997| 40.711261| 6.1| 1.0| 0.5| 0.0| 0.0| 7.6|{1, [[[-73.999779...|{1, [[[-74.00997,...|{5, [[[-80.0, 35....|
 CMT|2009-02-05 16:32:41|2009-02-05 16:52:21| 1| 4.4| -74.009162| 40.713183| null| -73.987602| 40.738736| 14.3| 0.0| null| 2.86| 0.0| 17.16|{1, [[[-74.009162...|{1, [[[-73.987602...|{5, [[[-80.0, 35....|
 VTS|2009-05-23 23:12:00|2009-05-23 23:27:00| 2| 4.62| -74.015995| 40.711183| null| -73.99806| 40.682233| 12.9| 0.5| null| 0.0| 0.0| 13.4|{1, [[[-74.015995...|{1, [[[-73.99806,...|{5, [[[-80.0, 35....|
 CMT|2010-02-22 14:24:15|2010-02-22 15:14:33| 1| 20.6| -74.010775| 40.70861| 1| -73.785869| 40.638801| 45.0| 0.0| 0.5| 7.51| 4.57| 57.58|{1, [[[-74.010775...|{1, [[[-73.785869...|{5, [[[-80.0, 35....|
 CMT|2009-05-13 23:00:55|2009-05-13 23:06:53| 1| 1.5| -73.998792| 40.734615| null| -73.993595| 40.751491| 6.1| 0.0| null| 0.0| 0.0| 6.1|{1, [[[-73.998792...|{1, [[[-73.993595...|{5, [[[-80.0, 35....|
 VTS|2009-02-13 20:44:00|2009-02-13 20:51:00| 1| 1.06| -74.01119| 40.715655| null| -74.016407| 40.70674| 5.3| 0.5| null| 0.0| 0.0| 5.8|{1, [[[-74.01119,...|{1, [[[-74.016407...|{5, [[[-80.0, 35....|
 CMT|2009-01-11 13:23:48|2009-01-11 13:31:00| 1| 1.6| -73.997643| 40.724385| null| -73.981072| 40.733372| 6.5| 0.0| null| 2.0| 0.0| 8.5|{1, [[[-73.997643...|{1, [[[-73.981072...|{5, [[[-80.0, 35....|
 CMT|2010-02-03 07:44:00|2010-02-03 07:58:12| 1| 4.7| -74.00716| 40.703949| 1| -73.9861| 40.740568| 13.3| 0.0| 0.5| 0.0| 0.0| 13.8|{1, [[[-74.00716,...|{1, [[[-73.9861

## Read from GeoJson, compute some basic geometry attributes

You've seen how Mosaic can create geometries from Spark native data types but it also provides functions to translate Well Known Text (WKT), Well Known Binary (WKB) and GeoJSON representations to Mosaic geometries.

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("dbfs:/FileStore/shared_uploads/stuart.lynn@databricks.com/NYC_Taxi_Zones.geojson")
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

+-------------+-----------+--------+--------------------+--------------------+
 borough|location_id|objectid| zone| geometry|
+-------------+-----------+--------+--------------------+--------------------+
 EWR| 1| 1| Newark Airport|{6, [[[-74.184452...|
 Queens| 2| 2| Jamaica Bay|{6, [[[-73.823375...|
 Bronx| 3| 3|Allerton/Pelham G...|{6, [[[-73.847926...|
 Manhattan| 4| 4| Alphabet City|{6, [[[-73.971774...|
Staten Island| 5| 5| Arden Heights|{6, [[[-74.174217...|
Staten Island| 6| 6|Arrochar/Fort Wad...|{6, [[[-74.063673...|
 Queens| 7| 7| Astoria|{6, [[[-73.904136...|
 Queens| 8| 8| Astoria Park|{6, [[[-73.923340...|
 Queens| 9| 9| Auburndale|{6, [[[-73.785024...|
 Manhattan| 24| 24| Bloomingdale|{6, [[[-73.959536...|
 Queens| 10| 10| Baisley Park|{6, [[[-73.783266...|
 Brooklyn| 11| 11| Bath Beach|{6, [[[-74.001098...|
 Manhattan| 12| 12| Battery Park|{6, [[[-74.015657...|
 Manhattan| 13| 13| Battery Park City|{6, [[[-74.012441...|
 Bronx| 18| 18| Bedford Park|{6, [[[-73.885139...|
 Brooklyn| 25| 25| Boerum Hill|{6, [[[-73.981552...|
 Brooklyn| 14| 14| Bay Ridge|{6, [[[-74.034073...|
 Queens| 15| 15|Bay Terrace/Fort ...|{6, [[[-73.777403...|
 Brooklyn| 22| 22| Bensonhurst West|{6, [[[-73.992549...|
Staten Island| 23| 23|Bloomfield/Emerso...|{6, [[[-74.195686...|
+-------------+-----------+--------+--------------------+--------------------+
only showing top 20 rows

Mosaic provides a number of functions for extracting the properties of geometries. Here are some that are relevant to Polygon geometries:

In [0]:
from mosaic import st_area, st_length
(
  geoJsonDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_length("geometry"))
  .select("geometry", "calculatedArea", "calculatedLength")
).show()

+--------------------+--------------------+--------------------+
 geometry| calculatedArea| calculatedLength|
+--------------------+--------------------+--------------------+
{6, [[[-74.184452...|7.823067885002562E-4| 0.1163574531886787|
{6, [[[-73.823375...|0.001422779097814599| 0.8431218810128789|
{6, [[[-73.847926...|3.144141568206508E-4| 0.08434110590105784|
{6, [[[-73.971774...| 7.94539194214528E-5| 0.03559982116907461|
{6, [[[-74.174217...|4.979574893632193E-4| 0.09214648985735088|
{6, [[[-74.063673...|4.046076598541714E-4| 0.12905753305870837|
{6, [[[-73.904136...|3.897879892739018...| 0.10741717112348065|
{6, [[[-73.923340...|2.658771690499072E-5|0.027590691195158792|
{6, [[[-73.785024...|3.384438031986125...| 0.0997840924710119|
{6, [[[-73.959536...|4.193691000471793E-5|0.034022318280805826|
{6, [[[-73.783266...|4.358238180810105...| 0.09983947941552027|
{6, [[[-74.001098...|2.029913532381591E-4| 0.07243934539937844|
{6, [[[-74.015657...|1.078595394871048...| 0.01759380786230351|
{6, [[[-74.012441...|4.874002713725957E-5| 0.04726120195723124|
{6, [[[-73.885139...|1.488501639473639E-4| 0.06979954985700013|
{6, [[[-73.981552...|1.241682673558662...| 0.04714581993191898|
{6, [[[-74.034073...|6.636590898190788E-4| 0.13622530962992194|
{6, [[[-73.777403...|4.588438269978550...| 0.1295148763926726|
{6, [[[-73.992549...|4.613649657988681...| 0.11844001178496774|
{6, [[[-74.195686...|0.002090822778761513| 0.3007595308122533|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
geoJsonDF.count()

Out[13]: 263

## Example point-in-poly with indexing

Mosaic has built-in support for the popular spatial indexing library, H3. The user has access to functions for generating point indices and the sets of indices covering polygons, allowing point-in-polygon joins to be transformed into deterministic SQL joins.

In [0]:
from mosaic import point_index

trips_with_geom = (
  trips
  .withColumn("pickup_h3", point_index(lng="pickup_longitude", lat="pickup_latitude", resolution=lit(10)))
  .withColumn("dropoff_h3", point_index(lng="dropoff_longitude", lat="dropoff_latitude", resolution=lit(10)))
)

trips_with_geom.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
 CMT|2009-05-16 23:09:38|2009-05-16 23:12:36| 1| 0.4| -73.993708| 40.724551| null| -74.001403| 40.730863| 3.7| 0.0| null| 0.0| 0.0| 3.7|{1, [[[-73.993708...|{1, [[[-74.001403...|{5, [[[-80.0, 35....|622236750717878271|622236750719582207|
 VTS|2010-04-30 05:50:00|2010-04-30 05:51:00| 1| 0.0| -74.183187| 40.68801| 5| -74.183145| 40.687972| 65.0| 0.0| 0.0| 20.0| 0.0| 85.0|{1, [[[-74.183187...|{1, [[[-74.183145...|{5, [[[-80.0, 35....|622236750416314367|622236750416314367|
 DDS|2009-05-29 17:25:37|2009-05-29 17:50:13| 1| 4.5| -74.002906| 40.728279| null| -73.954818| 40.767097| 15.3| 1.0| null| 4.05| 0.0| 20.35|{1, [[[-74.002906...|{1, [[[-73.954818...|{5, [[[-80.0, 35....|622236750718533631|622236723465322495|
 CMT|2009-12-28 18:19:23|2009-12-28 18:33:29| 1| 4.7| -73.997848| 40.726077| null| -73.988747| 40.777933| 13.3| 1.0| 0.5| 2.22| 0.0| 17.02|{1, [[[-73.997848...|{1, [[[-73.988747...|{5, [[[-80.0, 35....|622236750714863615|622236722234720255|
 CMT|2010-04-30 22:44:29|2010-04-30 22:57:05| 2| 2.5| -74.00514| 40.718808| 1| -73.986013| 40.751643| 9.3| 0.5| 0.5| 0.0| 0.0| 10.3|{1, [[[-74.00514,...|{1, [[[-73.986013...|{5, [[[-80.0, 35....|622236750713356287|622236723439960063|
 DDS|2009-05-03 00:12:28|2009-05-03 00:18:40| 1| 1.0| -73.991796| 40.725481| null| -73.977891| 40.72679| 5.3| 0.5| null| 0.0| 0.0| 5.8|{1, [[[-73.991796...|{1, [[[-73.977891...|{5, [[[-80.0, 35....|622236750714372095|622236723448152063|
 CMT|2009-01-31 01:36:33|2009-01-31 01:54:19| 1| 4.6| -73.990985| 40.734834| null| -73.957354| 40.726189| 15.0| 0.0| null| 0.0| 0.0| 15.0|{1, [[[-73.990985...|{1, [[[-73.957354...|{5, [[[-80.0, 35....|622236723428556799|622236723413909503|
 VTS|2010-02-09 00:39:00|2010-02-09 00:47:00| 1| 1.84| -74.004577| 40.723885| 1| -73.978537| 40.726283| 7.3| 0.5| 0.5| 0.0| 0.0| 8.3|{1, [[[-74.004577...|{1, [[[-73.978537...|{5, [[[-80.0, 35....|622236750706278399|622236723448152063|
 CMT|2010-02-05 17:00:49|2010-02-05 17:09:30| 1| 1.0| -73.999779| 40.718208| 1| -74.00997| 40.711261| 6.1| 1.0| 0.5| 0.0| 0.0| 7.6|{1, [[[-73.999779...|{1, [[[-74.00997,...|{5, [[[-80.0, 35....|622236750709424127|622236750648508415|
 CMT|2009-02-05 16:32:41|2009-02-05 16:52:21| 1| 4.4| -74.009162| 40.713183| null| -73.987602| 40.738736| 14.3| 0.0| null| 2.86| 0.0| 17.16|{1, [[[-74.009162...|{1, [[[-73.987602...|{5, [[[-80.0, 35....|622236750648606719|622236723426820095|
 VTS|2009-05-23 23:12:00|2009-05-23 23:27:00| 2| 4.62| -74.015995| 40.711183| null| -73.99806| 40.682233| 12.9| 0.5| null| 0.0| 0.0| 13.4|{1, [[[-74.015995...|{1, [[[-73.99806,...|{5, [[[-80.0, 35....|622236750652506111|622236750655586303|
 CMT|2010-02-22 14:24:15|2010-02-22 15:14:33| 1| 20.6| -74.010775| 40.70861| 1| -73.785869| 40.638801| 45.0| 0.0| 0.5| 7.51| 4.57| 57.58|{1, [[[-74.010775...|{1, [[[-73.785869...|{5, [[[-80.0, 35....|622236750647459839|622236735761317887|
 CMT|2009-05-13 23:00:55|2009-05-13 23:06:53| 1| 1.5| -73.998792| 40.734615| null| -73.993595| 40.751491| 6.1| 0.0| null| 0.0| 0.0| 6.1|{1, [[[-73.998792...|{1, [[[-73.993595...|{5, [[[-80.0, 35....|622236723433

In [0]:
from mosaic import polyfill

neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", explode(polyfill("geometry", lit(10))).alias("h3"))
  .drop("geometry")
)

neighbourhoods.show()

+-------+-----------+--------+--------------------+------------------+
borough|location_id|objectid| zone| h3|
+-------+-----------+--------+--------------------+------------------+
 Queens| 207| 207|Saint Michaels Ce...|622236723991183359|
 Queens| 207| 207|Saint Michaels Ce...|622236723950878719|
 Queens| 207| 207|Saint Michaels Ce...|622236723989315583|
 Queens| 207| 207|Saint Michaels Ce...|622236723991216127|
 Queens| 207| 207|Saint Michaels Ce...|622236723988201471|
 Queens| 207| 207|Saint Michaels Ce...|622236723991248895|
 Queens| 207| 207|Saint Michaels Ce...|622236723990134783|
 Queens| 207| 207|Saint Michaels Ce...|622236723990167551|
 Queens| 207| 207|Saint Michaels Ce...|622236723950223359|
 Queens| 207| 207|Saint Michaels Ce...|622236723988299775|
 Queens| 207| 207|Saint Michaels Ce...|622236723951009791|
 Queens| 207| 207|Saint Michaels Ce...|622236723950256127|
 Queens| 207| 207|Saint Michaels Ce...|622236723988332543|
 Queens| 207| 207|Saint Michaels Ce...|622236723951042559|
 Queens| 207| 207|Saint Michaels Ce...|622236723991379967|
 Queens| 207| 207|Saint Michaels Ce...|622236723950682111|
 Queens| 207| 207|Saint Michaels Ce...|622236723950714879|
 Queens| 207| 207|Saint Michaels Ce...|622236723991052287|
 Queens| 207| 207|Saint Michaels Ce...|622236723950747647|
 Queens| 207| 207|Saint Michaels Ce...|622236723951140863|
+-------+-----------+--------+--------------------+------------------+
only showing top 20 rows

In [0]:
joined_df = trips_with_geom.alias("t").join(neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
joined_df.count()

Out[16]: 4942629

## Mosaic spatial join optimizations

Mosaic provides easy access to the optimized spatial join technique described in [this](https://databricks.com/blog/2021/10/11/efficient-point-in-polygon-joins-via-pyspark-and-bng-geospatial-indexing.html) blog post.

In [0]:
from mosaic import mosaic_explode

mosaic_neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", mosaic_explode("geometry", lit(10)))
  .drop("geometry")
)

mosaic_neighbourhoods.show()

+-------+-----------+--------+--------------------+-------+------------------+--------------------+
borough|location_id|objectid| zone|is_core| h3| wkb|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950682111| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950845951| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723951009791| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988332543| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950780415| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991085055| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991248895| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950714879| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991183359| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988299775| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950747647| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991052287| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991216127| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991379967| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723951140863| null|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723990167551|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723991117823|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950288895|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723988365311|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950878719|[01 03 00 00 00 0...|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
only showing top 20 rows

Now the two datasets can be joined first on H3 index, with any false positives removed through a contains filter on a much simpler geometry.

In [0]:
mosaic_joined_df = (
  trips_with_geom.alias("t")
  .join(mosaic_neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
  .where(
    ~col("is_core") | 
    st_contains("wkb", "pickup_geom")
  )
)

mosaic_joined_df.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3| borough|location_id|objectid| zone|is_core| h3| wkb|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
 CMT|2009-05-16 23:09:38|2009-05-16 23:12:36| 1| 0.4| -73.993708| 40.724551| null| -74.001403| 40.730863| 3.7| 0.0| null| 0.0| 0.0| 3.7|{1, [[[-73.993708...|{1, [[[-74.001403...|{5, [[[-80.0, 35....|622236750717878271|622236750719582207|Manhattan| 144| 144| Little Italy/NoLiTa| false|622236750717878271|[01 03 00 00 00 0...|
 CMT|2009-05-16 23:09:38|2009-05-16 23:12:36| 1| 0.4| -73.993708| 40.724551| null| -74.001403| 40.730863| 3.7| 0.0| null| 0.0| 0.0| 3.7|{1, [[[-73.993708...|{1, [[[-74.001403...|{5, [[[-80.0, 35....|622236750717878271|622236750719582207|Manhattan| 114| 114|Greenwich Village...| false|622236750717878271|[01 03 00 00 00 0...|
 DDS|2009-05-29 17:25:37|2009-05-29 17:50:13| 1| 4.5| -74.002906| 40.728279| null| -73.954818| 40.767097| 15.3| 1.0| null| 4.05| 0.0| 20.35|{1, [[[-74.002906...|{1, [[[-73.954818...|{5, [[[-80.0, 35....|622236750718533631|622236723465322495|Manhattan| 249| 249| West Village| false|622236750718533631|[01 03 00 00 00 0...|
 DDS|2009-05-29 17:25:37|2009-05-29 17:50:13| 1| 4.5| -74.002906| 40.728279| null| -73.954818| 40.767097| 15.3| 1.0| null| 4.05| 0.0| 20.35|{1, [[[-74.002906...|{1, [[[-73.954818...|{5, [[[-80.0, 35....|622236750718533631|622236723465322495|Manhattan| 125| 125| Hudson Sq| false|622236750718533631|[01 03 00 00 00 0...|
 CMT|2009-12-28 18:19:23|2009-12-28 18:33:29| 1| 4.7| -73.997848| 40.726077| null| -73.988747| 40.777933| 13.3| 1.0| 0.5| 2.22| 0.0| 17.02|{1, [[[-73.997848...|{1, [[[-73.988747...|{5, [[[-80.0, 35....|622236750714863615|622236722234720255|Manhattan| 211| 211| SoHo| false|622236750714863615|[01 03 00 00 00 0...|
 CMT|2009-12-28 18:19:23|2009-12-28 18:33:29| 1| 4.7| -73.997848| 40.726077| null| -73.988747| 40.777933| 13.3| 1.0| 0.5| 2.22| 0.0| 17.02|{1, [[[-73.997848...|{1, [[[-73.988747...|{5, [[[-80.0, 35....|622236750714863615|622236722234720255|Manhattan| 114| 114|Greenwich Village...| false|622236750714863615|[01 03 00 00 00 0...|
 DDS|2009-05-03 00:12:28|2009-05-03 00:18:40| 1| 1.0| -73.991796| 40.725481| null| -73.977891| 40.72679| 5.3| 0.5| null| 0.0| 0.0| 5.8|{1, [[[-73.991796...|{1, [[[-73.977891...|{5, [[[-80.0, 35....|622236750714372095|622236723448152063|Manhattan| 79| 79| East Village| false|622236750714372095|[01 03 00 00 00 0...|
 DDS|2009-05-03 00:12:28|2009-05-03 00:18:40| 1| 1.0| -73.991796| 40.725481| null| -73.977891| 40.72679| 5.3| 0.5| null| 0.0| 0.0| 5.8|{1, [[[-73.991796...|{1, [[[-73.977891...|{5, [[[-80.0, 35....|622236750714372095|622236723448152063|Manhattan| 114| 114|Greenwich Village...| false|622236750714372095|[01 03 00 00 00 0...|
 CMT|2009-01-31 01:36:33|2009-01-31 01:54:19| 1| 4.6| -73.990985| 40.734834| null| -73.957354| 40.726189| 15.0| 0.0| null| 0.0| 0.0| 15.0|{1, [[[-73.990985...|{1, [[[-73.957354...|{5, [[[-80.0, 35....|622236723428556799|6222367234139095